# Planet Training Data Formatting Script

Use and modify this script to work with reformatting training data classifications used to feed the kelp classification neural network.
This script is set up to work with/access files locally stored on your computer (I never figured out how to redirect the current directory to server paths), as opposed to the server which most of the files are usually stored on.

To work this way, simply make a copy of the folders with the files that you're working with onto your local desktop, and once you get the final outputs
that you're satisfied with, move those back onto the server.


In [ ]:
## Set up Packages ## 
import os
import arcpy
from arcpy import management
from arcpy import sa

In [ ]:
## Set up your current directory
# this method uses arcpy instead of os package, but either could work
arcpy.env.workspace = "C:/Users/mleung/Desktop/trainingData_planet8band/monterey/rerun/finalOutputs"

walk = arcpy.da.Walk(datatype="RasterDataset")

for dir_path, dir_names, file_names in walk:
    for filename in file_names:
        print(os.path.join(dir_path, filename))

fileList = arcpy.ListFiles("*.tif")
print(fileList)


## Copy Raster Tool
Run the next cell if you are trying to automate the Copy Raster tool, for example, to loop through rasters in a folder that all need to have a NoData value changed.

In [ ]:
## COPY RASTER ##
fileList = arcpy.ListFiles("*.tif")

for raster in arcpy.ListFiles("*.tif"):
    # get the file name without extension (replaces the %Name% variable from ModelBuilder)
    fileName = os.path.splitext(raster)[0]
    # define the output file
    rastFile = fileName + "_final.tif" 
    # run the copy raster tool
    arcpy.management.CopyRaster(raster,
                                rastFile,
                                '', 
                                '',
                                3,
                                'NONE',
                                'NONE',
                                '8_BIT_UNSIGNED',
                                'NONE',
                                'NONE', # color map to RGB
                                'TIFF',
                                'NONE',
                                'CURRENT_SLICE',
                                'NO_TRANSPOSE')

## Reclassify Tool
Run the next cell if you are trying to automate the Reclassify (Spatial analyst) tool. For example, to loop through rasters in a folder that all need to have their classes redefined to different values.

Refer to https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/reclassify.htm for code examples for the reclassify() function's parameters 

In [ ]:
## RECLASSIFY ##
fileList = arcpy.ListFiles("*.tif")

for raster in arcpy.ListFiles("*.tif"):
    # get the file name without extension (replaces the %Name% variable from ModelBuilder)
    fileName = os.path.splitext(raster)[0]
    # define the output file
    rastFile = fileName + "_reclass.tif" 
    # run the copy raster tool
    arcpy.spatialAnalyst.reclassify(raster,
                                "Value",
                                RemapValue([1,2], [3,2]), #change your remap values here within the square brackets
                                )
    outReclass.save(rastFile)